# *Kiểm tra repository của một bài báo khoa học*

**Môn học**: Lập trình cho Khoa học dữ liệu

**Thông tin sinh viên**

Họ tên sinh viên: Nguyễn Thị Ánh Tuyết

Mã số sinh viên: 20120422

## Import các thư viện cần thiết

In [1]:
import re
import threading
from io import BytesIO
import PyPDF2                   # thư viện đọc file PDF
import requests
from bs4 import BeautifulSoup

In [2]:
# pdf_name = 'Zero-Shot Image Restoration Using Denoising Diffusion Null-Space Model'
pdf_name = input('Nhap ten bai bao: ')

Nhap ten bai bao: Zero-Shot Image Restoration Using Denoising Diffusion Null-Space Model


## Tìm các bài pdf trên google chứa tên giống hoặc gần giống với bài báo cần tìm

In [3]:
search_response = requests.get(f'https://google.com/search?q={pdf_name + " filetype:pdf"}').text
soup = BeautifulSoup(search_response, 'html.parser')

In [4]:
valid_a = []
for a in soup.select('a'):
    if '[PDF]' in str(a) and a['href'].startswith('/url?q='):
        valid_a.append(a)
urls = sum((re.findall(r'/url\?q=([^&]+)', a['href']) for a in valid_a), start=[])
urls

['https://openreview.net/pdf%3Fid%3DmRieQgMtNTQ',
 'https://arxiv.org/pdf/2212.00490',
 'https://openaccess.thecvf.com/content/CVPR2021/papers/Kar_Zero-Shot_Single_Image_Restoration_Through_Controlled_Perturbation_of_Koschmieders_Model_CVPR_2021_paper.pdf',
 'https://web.eecs.umich.edu/~fessler/book/c-restore.pdf',
 'https://core.ac.uk/download/pdf/42390023.pdf',
 'https://www.robots.ox.ac.uk/~vedaldi/assets/pubs/ulyanov20deep.pdf',
 'https://users.soe.ucsc.edu/~milanfar/publications/dissertations/Thesis_final_Amin.pdf',
 'https://eccv2022.ecva.net/files/2021/12/ECCV_2022_MainConference_ProgramGuide_Final_full.pdf']

## Truy cập vào các link file pdf để lấy nội dung file pdf 

- Dictionary `contents` chứa key là link file, value là nội dung file


In [5]:
contents = {}


def thread_run(url):
    try:
        response = requests.get(url, timeout=30)
    except requests.Timeout:
        print(f"Get {url} timeout (more than 60)")
        return

    if response.status_code != 404:
        contents[url] = response.content
        print(f"Get {url} done")
    else:
        print(f"Get {url} error - 404")


threads = [threading.Thread(target=thread_run, args=(url,)) for url in urls]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()

print(f'Số lượng trang dạng ".pdf" hợp lệ: {len(contents)}')

Get https://openreview.net/pdf%3Fid%3DmRieQgMtNTQ error - 404
Get https://core.ac.uk/download/pdf/42390023.pdf done
Get https://web.eecs.umich.edu/~fessler/book/c-restore.pdf done
Get https://eccv2022.ecva.net/files/2021/12/ECCV_2022_MainConference_ProgramGuide_Final_full.pdf done
Get https://www.robots.ox.ac.uk/~vedaldi/assets/pubs/ulyanov20deep.pdf done
Get https://users.soe.ucsc.edu/~milanfar/publications/dissertations/Thesis_final_Amin.pdf done
Get https://openaccess.thecvf.com/content/CVPR2021/papers/Kar_Zero-Shot_Single_Image_Restoration_Through_Controlled_Perturbation_of_Koschmieders_Model_CVPR_2021_paper.pdf done
Get https://arxiv.org/pdf/2212.00490 done
Số lượng trang dạng ".pdf" hợp lệ: 7


## Tìm kiếm và hiển thị các link github tìm thấy được trong nội dung các file pdf
Link Github được trích ra từ file pdf, do đó với mỗi file pdf có thể có nhiều link Github. Bài làm:
- Sử dụng thư viện PyPDF2 để đọc file pdf
- Tìm kiếm đường link Github với chuỗi bắt đầu là 'https://github.com'
- Đánh dấu tìm được link Github trong quá trình tìm kiếm.

    

In [6]:
pdf_link = {}
having_github_link = False

for url, pdf_content in contents.items():
    url: str
    pdf_content: bytes

    try:
        pdf_reader = PyPDF2.PdfReader(BytesIO(pdf_content))
        pages = list(pdf_reader.pages) 
    except Exception:
        continue

    for page in pages:
        text = page.extract_text().replace('\n', '')
        if match := re.findall(r'(https://github\.com.*?)[ .]', text):
            link = match[0]
            if not having_github_link: 
                having_github_link = True
            pdf_link[url] = pdf_link.get(url, []) + [link]


Lúc này, có thể link Github được lấy chưa phải là link Github thực tế, nên em kiểm tra và sửa lại bằng cách:
- Tìm tất cả các link repository của username đó trong Github.
- Kiểm tra tên của repository cần tìm so với tên của repository thực tế có trong Github người đó:
    - Nếu gần giống thì khả năng là do quá trình lấy bị thừa hoặc thiếu ==> thay thế bằng link của repo thực tế.

In [7]:
# Get all public repo from user name
def get_all_Github_repo(username):
    repos_name = []
    resonse = requests.get(f'https://api.github.com/users/{username}/repos')
    repos = resonse.json()
    for repo in repos:
        repos_name.append(repo['name'])
#         print(repo['name'])
    return repos_name

In [8]:
# Recheck and replace some incorrect Github link 
def recheck(pdf_link):
    for key in pdf_link:
        for i, link in enumerate(pdf_link[key]):
            user_name = re.search(r'https://github\.com/(.*)/', link).group(1)
            repo_name = re.search(r'https://github\.com/.*/(.*)', link).group(1)
#             print(user_name)
            all_repo_name = get_all_Github_repo(user_name)
            
            for repo in all_repo_name:
                if repo in repo_name or repo_name in repo:
                    pdf_link[key][i] = 'https://github.com/' + user_name + '/' + repo
#                     print(pdf_link[key][i])
                    break
    return pdf_link

### Kết quả
- Kết quả sẽ gồm các link github tìm thấy được trong nội dung file pdf hoặc "There is no Github source!" nếu không tìm thấy được link Github cho bài báo.
- Kết quả khi tìm được sẽ hiển thị dạng:
    - "Link pdf":
        - "Link Github: ..."
        - "Link Github: ..."
    - ...

In [9]:
def print_link(data : dict):
    for key in data:
        print(f'Link pdf: {key}')
        print('     Link Github:', end = '')
        print('\n     ---> '.join([''] + data[key]))

In [10]:
if having_github_link: 
    pdf_link = recheck(pdf_link)
    print_link(pdf_link)
else:
    print('There is no Github source!')

Link pdf: https://www.robots.ox.ac.uk/~vedaldi/assets/pubs/ulyanov20deep.pdf
     Link Github:
     ---> https://github.com/gfacciol/bm3d
Link pdf: https://arxiv.org/pdf/2212.00490
     Link Github:
     ---> https://github.com/wyhuai/DDNM
